In [14]:
from datetime import datetime
import os
import itertools
import pandas as pd
import numpy as np
import requests
import json
import pickle
from tqdm import tqdm

os.makedirs("../data/", exist_ok=True)

First download "Solar home half-hour data - 1 July 2010 to 30 June 2011" from https://www.ausgrid.com.au/Industry/Our-Research/Data-to-share/Solar-home-electricity-data, unzip and place "2010-2011 Solar home electricity data.csv" in the data folder

In [15]:
solar_home_data = pd.read_csv("../data/2010-2011 Solar home electricity data.csv", header=1)

Each customer has either two or three types of recordings for each day of the year. The recordings correspond to the consumption categories.

In [16]:
solar_home_data

,Customer,Generator Capacity,Postcode,Consumption Category,date,0:30,1:00,1:30,2:00,2:30,...,19:30,20:00,20:30,21:00,21:30,22:00,22:30,23:00,23:30,0:00
0,1,3.78,2076,GC,1-Jul-10,0.303,0.471,0.083,0.121,0.361,...,0.495,0.540,0.406,0.543,0.495,0.216,0.378,0.128,0.078,0.125
1,1,3.78,2076,CL,1-Jul-10,1.250,1.244,1.256,0.744,0.019,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.075
2,1,3.78,2076,GG,1-Jul-10,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,1,3.78,2076,GC,2-Jul-10,0.116,0.346,0.122,0.079,0.120,...,0.262,1.100,1.012,0.817,0.526,0.335,0.402,0.142,0.120,0.111
4,1,3.78,2076,CL,2-Jul-10,1.238,1.238,1.256,1.250,0.169,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269730,300,3.36,2086,CL,29-Jun-11,2.136,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2.490
269731,300,3.36,2086,GG,29-Jun-11,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
269732,300,3.36,2086,GC,30-Jun-11,1.306,0.851,0.113,0.151,0.104,...,0.948,0.917,0.872,0.832,0.863,0.789,0.280,0.224,0.259,0.774
269733,300,3.36,2086,CL,30-Jun-11,0.904,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2.528


Convert to net energy

In [17]:
has_cl = solar_home_data.Customer.unique()[
    [((solar_home_data.Customer == c) & (solar_home_data['Consumption Category'] == "CL")).any() for c in solar_home_data.Customer.unique()]
]
time_columns = solar_home_data.columns[5:]

In [18]:
gc_data = solar_home_data[(solar_home_data['Consumption Category'] == "GC") & solar_home_data.Customer.isin(has_cl)].reset_index(drop=True)
cl_data = solar_home_data[(solar_home_data['Consumption Category'] == "CL") & solar_home_data.Customer.isin(has_cl)].reset_index(drop=True)
gg_data = solar_home_data[(solar_home_data['Consumption Category'] == "GG") & solar_home_data.Customer.isin(has_cl)].reset_index(drop=True)

In [19]:
gc_data.loc[:, time_columns] = cl_data[time_columns] + gc_data[time_columns]
consumption_data = gc_data.drop(columns=["Consumption Category"])
consumption_data

,Customer,Generator Capacity,Postcode,date,0:30,1:00,1:30,2:00,2:30,3:00,...,19:30,20:00,20:30,21:00,21:30,22:00,22:30,23:00,23:30,0:00
0,1,3.78,2076,1-Jul-10,1.553,1.715,1.339,0.865,0.380,1.329,...,0.495,0.540,0.406,0.543,0.495,0.216,0.378,0.128,0.078,1.200
1,1,3.78,2076,2-Jul-10,1.354,1.584,1.378,1.329,0.289,0.701,...,0.262,1.100,1.012,0.817,0.526,0.335,0.402,0.142,0.120,1.199
2,1,3.78,2076,3-Jul-10,1.980,2.633,2.042,1.887,0.729,1.674,...,0.441,0.387,0.274,0.782,0.519,0.225,0.123,0.157,0.390,1.205
3,1,3.78,2076,4-Jul-10,1.399,1.333,1.393,1.321,0.123,1.300,...,1.087,0.992,0.462,0.379,0.505,0.586,0.269,0.252,0.295,1.313
4,1,3.78,2076,5-Jul-10,1.545,1.725,1.645,1.316,0.122,1.203,...,0.904,0.945,0.385,0.295,1.034,0.169,0.122,0.135,0.079,1.453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50730,300,3.36,2086,26-Jun-11,3.236,3.135,2.011,0.529,0.434,0.396,...,1.576,1.094,0.532,0.550,0.568,0.491,0.471,0.416,0.451,3.301
50731,300,3.36,2086,27-Jun-11,3.570,3.098,1.665,0.099,0.140,0.105,...,1.176,0.709,0.794,0.696,0.614,0.553,0.596,0.498,0.260,3.434
50732,300,3.36,2086,28-Jun-11,3.344,2.754,0.105,0.110,0.090,0.114,...,1.003,0.880,0.828,0.858,0.871,0.637,0.158,0.139,0.170,3.475
50733,300,3.36,2086,29-Jun-11,3.028,1.048,0.250,0.132,0.100,0.134,...,1.116,1.276,1.163,1.178,1.076,1.094,0.990,0.500,0.514,3.392


In [20]:
generation_data = gg_data.drop(columns=["Consumption Category"])
generation_data

,Customer,Generator Capacity,Postcode,date,0:30,1:00,1:30,2:00,2:30,3:00,...,19:30,20:00,20:30,21:00,21:30,22:00,22:30,23:00,23:30,0:00
0,1,3.78,2076,1-Jul-10,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,3.78,2076,2-Jul-10,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,3.78,2076,3-Jul-10,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,3.78,2076,4-Jul-10,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,3.78,2076,5-Jul-10,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50730,300,3.36,2086,26-Jun-11,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50731,300,3.36,2086,27-Jun-11,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50732,300,3.36,2086,28-Jun-11,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50733,300,3.36,2086,29-Jun-11,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
gc_data = solar_home_data[(solar_home_data['Consumption Category'] == "GC") & ~solar_home_data.Customer.isin(has_cl)].reset_index(drop=True)
gg_data = solar_home_data[(solar_home_data['Consumption Category'] == "GG") & ~solar_home_data.Customer.isin(has_cl)].reset_index(drop=True)
consumption_data = pd.concat((consumption_data, gc_data.drop(columns=["Consumption Category"])))
generation_data = pd.concat((generation_data, gg_data.drop(columns=["Consumption Category"])))

Get rid of half hour intervals

In [22]:
def combine_half_hour(data):
    time_columns = data.columns[4:]
    for half_hour, hour in zip(time_columns[::2], time_columns[1::2]):
        data[hour] += data[half_hour]
    data = data.drop(columns=time_columns[::2])
    return data


consumption_data = combine_half_hour(consumption_data)
generation_data = combine_half_hour(generation_data)

Make long form

In [23]:
def to_long_form(data):
    id_columns = data.columns[:4].tolist()
    time_columns = data.columns[4:]
    data = pd.melt(data, id_vars=id_columns, value_vars=time_columns)
    return data

consumption_data = to_long_form(consumption_data)
generation_data = to_long_form(generation_data)

Clean up the formatting of the rows and save the processed dataset

In [24]:
def format_data(data, value_name):
    data = data.rename(columns={"value": value_name, "date": "Datetime"})
    data['Datetime'] += " " + data['variable']
    data = data.drop(columns='variable')
    data['Datetime'] = data['Datetime'].map(lambda x: datetime.strptime(x, "%d-%b-%y %H:%M").isoformat(timespec="minutes"))
    return data

consumption_data = format_data(consumption_data, "Consumed Energy")
generation_data = format_data(generation_data, "Generated Energy")

In [26]:
solar_home_data = consumption_data.merge(generation_data)
solar_home_data.to_csv("../data/solar_home_2010-2011.csv", index=False)
solar_home_data

,Customer,Generator Capacity,Postcode,Datetime,Consumed Energy,Generated Energy
0,1,3.78,2076,2010-07-01T01:00,3.268,0.0
1,1,3.78,2076,2010-07-02T01:00,2.938,0.0
2,1,3.78,2076,2010-07-03T01:00,4.613,0.0
3,1,3.78,2076,2010-07-04T01:00,2.732,0.0
4,1,3.78,2076,2010-07-05T01:00,3.270,0.0
...,...,...,...,...,...,...
2627995,299,1.00,2076,2011-06-26T00:00,0.162,0.0
2627996,299,1.00,2076,2011-06-27T00:00,0.784,0.0
2627997,299,1.00,2076,2011-06-28T00:00,0.197,0.0
2627998,299,1.00,2076,2011-06-29T00:00,1.245,0.0


 Get postal data for using the weather data. First download and extract the zip file from https://download.geonames.org/export/zip/AU.zip and place "AU.txt" in the data folder

In [28]:
postcode_data = pd.read_csv(
    "../data/AU.txt",
    sep='\t',
    names=["country code","postal code","place name","admin name1","admin code1","admin name2","admin code2","admin name3","admin code3","latitude","longitude","accuracy"]
)
postcode_data = postcode_data.drop(postcode_data[postcode_data['postal code'].duplicated()].index)  # we will use just one of the postcodes
postcode_data

,country code,postal code,place name,admin name1,admin code1,admin name2,admin code2,admin name3,admin code3,latitude,longitude,accuracy
0,AU,200,Australian National University,Australian Capital Territory,ACT,CANBERRA,NaN,NaN,NaN,-35.2777,149.1189,1.0
1,AU,221,Barton,Australian Capital Territory,ACT,NaN,NaN,NaN,NaN,-35.3049,149.1412,4.0
2,AU,2540,Jervis Bay,Australian Capital Territory,ACT,NEW CNTRY WEST,NaN,NaN,NaN,-35.1499,150.6969,4.0
5,AU,2600,Russell,Australian Capital Territory,ACT,CANBERRA,NaN,NaN,NaN,-35.2991,149.1515,4.0
17,AU,2601,Canberra,Australian Capital Territory,ACT,CANBERRA,NaN,NaN,NaN,-35.2835,149.1281,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
16868,AU,6989,Maddington,Western Australia,WA,TANGNEY,NaN,NaN,NaN,-32.0500,115.9833,4.0
16869,AU,6990,Gosnells,Western Australia,WA,TANGNEY,NaN,NaN,NaN,-32.0810,116.0054,4.0
16870,AU,6991,Kelmscott,Western Australia,WA,TANGNEY,NaN,NaN,NaN,-32.1243,116.0259,4.0
16871,AU,6992,Armadale,Western Australia,WA,TANGNEY,NaN,NaN,NaN,-32.1461,116.0093,4.0


Then we download the weather data from https://open-meteo.com and categorise by postcode

In [29]:
weather_data = {
    "structure": {"postcode": {"time": ["temperature", "precipitation", "cloudcover"]}},
    "hourly units": {'time': 'iso8601', 'temperature': 'Celcius', 'precipitation': 'mm', 'cloudcover': '%'},
}

for postcode in (pbar := tqdm(solar_home_data.Postcode.unique())):
    pbar.set_postfix_str(f"{postcode=}")
    postcode_row = postcode_data[postcode == postcode_data['postal code']]
    latitude, longitude = postcode_row.latitude.item(), postcode_row.longitude.item()
    r = requests.get(f"https://archive-api.open-meteo.com/v1/era5?latitude={latitude}&longitude={longitude}&start_date=2010-07-01&end_date=2011-06-30&hourly=temperature_2m,precipitation,cloudcover&timezone=Australia%2FSydney")
    if r.ok:
        postcode_weather = r.json()
        weather_data[str(postcode)] = {time: other_data for time, *other_data in zip(*[v for v in postcode_weather['hourly'].values()])}
    else:
        tqdm.write(f"Postcode {postcode} weather data request failed")

with open(f"../data/weather.json", 'w') as f:
    json.dump(weather_data, f)

100%|██████████| 100/100 [02:48<00:00,  1.69s/it, postcode=2216]


For the final steps of processing, we first divide the clients into regions based on postcode

In [30]:
postcodes = solar_home_data.Postcode.unique()
_, bins = np.histogram(postcodes, 5)
postcode_region_map = {postcode: (i - 1).item() for postcode, i in zip(postcodes, np.digitize(postcodes, bins))}
customers = solar_home_data.Customer.unique()
customer_region_map = {}
for customer in customers:
    region = postcode_region_map[solar_home_data.query(f"`Customer` == {customer}").Postcode.unique().item()]
    customer_region_map[str(customer)] = region

with open("../data/customer_regions.json", 'w') as f:
    json.dump(customer_region_map, f)

Then we sort and format the rest of the data for use in the machine learning model

In [33]:
sorted_full_data = {}
for customer in tqdm(solar_home_data.Customer.unique()):
    customer_data = solar_home_data.query(f"`Customer` == {customer}")
    postcode_weather_data = weather_data[str(customer_data.Postcode.unique().item())]
    net_energy_data = {
        time: [con_energy, gen_energy]
        for time, con_energy, gen_energy in zip(customer_data.Datetime, customer_data["Consumed Energy"], customer_data["Generated Energy"])
    }

    idx = np.argsort([datetime.fromisoformat(k).timestamp() for k in net_energy_data.keys()])
    sorted_energy = np.array(list(net_energy_data.values()))[idx]
    idx = np.argsort([datetime.fromisoformat(k).timestamp() for k in postcode_weather_data.keys()])
    sorted_weather_data = np.array(list(postcode_weather_data.values()))[idx]

    sorted_full_data[customer] = np.concatenate((sorted_energy.reshape(-1, 2), sorted_weather_data), axis=1)

100%|██████████| 300/300 [00:06<00:00, 47.56it/s]


In [36]:
with open("../data/solar_home_data.pkl", 'wb') as f:
    pickle.dump(sorted_full_data, f)

An example sampling method:

In [49]:
def sample(batch_size=32, customer=1):
    idx = np.random.randint(24, len(sorted_full_data[customer]), size=batch_size)
    expanded_idx = np.array([np.arange(i - 24, i - 1) for i in idx])
    return sorted_full_data[customer][expanded_idx].reshape(batch_size, -1), sorted_full_data[customer][idx, :2]

In [50]:
sample()

(array([[1.586e+00, 0.000e+00, 1.670e+01, ..., 1.750e+01, 0.000e+00,
         6.900e+01],
        [2.220e+00, 0.000e+00, 1.510e+01, ..., 1.580e+01, 0.000e+00,
         3.300e+01],
        [1.547e+00, 0.000e+00, 1.810e+01, ..., 1.710e+01, 2.000e-01,
         1.000e+02],
        ...,
        [2.090e-01, 0.000e+00, 1.460e+01, ..., 1.390e+01, 1.000e-01,
         5.300e+01],
        [1.892e+00, 2.500e-02, 1.510e+01, ..., 1.860e+01, 2.000e-01,
         3.500e+01],
        [2.150e-01, 6.250e-01, 1.790e+01, ..., 2.100e+01, 0.000e+00,
         9.500e+01]]),
 array([[1.273, 0.   ],
        [2.659, 0.   ],
        [1.799, 0.   ],
        [0.594, 1.125],
        [1.891, 0.   ],
        [0.579, 0.376],
        [1.648, 0.   ],
        [0.557, 0.344],
        [1.785, 0.   ],
        [0.719, 2.007],
        [1.674, 0.   ],
        [0.498, 0.   ],
        [0.713, 1.969],
        [1.533, 0.   ],
        [0.336, 1.319],
        [0.612, 0.   ],
        [0.269, 1.275],
        [0.44 , 0.606],
        [2.82

Or get full processed data

In [51]:
def get_customer_data(customer=1):
    idx = np.arange(24, len(sorted_full_data[customer]))
    expanded_idx = np.array([np.arange(i - 24, i - 1) for i in idx])
    return sorted_full_data[customer][expanded_idx].reshape(len(sorted_full_data[customer]) - 24, -1), sorted_full_data[customer][idx, :2]

In [52]:
get_customer_data()

(array([[ 1.278,  0.   ,  6.3  , ...,  7.3  ,  0.   , 29.   ],
        [ 3.268,  0.   ,  5.9  , ...,  7.3  ,  0.   , 30.   ],
        [ 2.204,  0.   ,  5.5  , ...,  7.2  ,  0.   , 30.   ],
        ...,
        [ 0.397,  0.   , 12.9  , ..., 12.4  ,  0.   , 43.   ],
        [ 0.597,  0.   , 12.5  , ..., 12.1  ,  0.   , 90.   ],
        [ 0.387,  0.   , 12.1  , ..., 11.9  ,  0.   , 86.   ]]),
 array([[1.319, 0.   ],
        [2.938, 0.   ],
        [2.707, 0.   ],
        ...,
        [0.429, 0.   ],
        [0.601, 0.   ],
        [1.143, 0.   ]]))